In [5]:
#returns both dataframes - prices and grades separately in the long format:

import pandas as pd
import json
import os

def load_and_flatten_data(sets_dir):
    all_prices = []
    all_population_reports = []

    all_sets = [d for d in os.listdir(sets_dir) if os.path.isdir(os.path.join(sets_dir, d))]
    for set_folder in all_sets:
        set_path = os.path.join(sets_dir, set_folder)
        json_files = [f for f in os.listdir(set_path) if f.endswith('.json')]

        for json_file in json_files:
            with open(os.path.join(set_path, json_file), 'r', encoding='utf-8') as file:
                data = json.load(file)
                set_name = data['set_name'].strip()
                card_name = data['card_name'].strip()

                # Check and Flatten prices_data
                if data['prices_data'] and isinstance(data['prices_data'], list):
                    for price_data in data['prices_data']:
                        if price_data is not None:  # Check if price_data is not None
                            price_data['set_name'] = set_name
                            price_data['card_name'] = card_name
                            all_prices.append(price_data)
                else:
                    # Add a record indicating no price data available
                    all_prices.append({
                        'Date': None,
                        'Grade': None,
                        'Price': None,
                        'Sale Type': None,
                        'set_name': set_name,
                        'card_name': card_name
                    })

                # Flatten population_report
                for report in data['population_report']:
                    all_population_reports.append({
                        'set_name': set_name,
                        'card_name': card_name,
                        'grade': report[0],
                        'count': report[1]
                    })

    return pd.DataFrame(all_prices), pd.DataFrame(all_population_reports)

# Example usage
sets_dir = 'sets'  # Replace with your actual directory
prices_df, pop_reports_df = load_and_flatten_data(sets_dir)

prices_df, pop_reports_df

(              Date Grade   Price   Sale Type                   set_name   
 0       2023-10-30     9  $12.50     Auction  2021 Pokemon Celebrations  \
 1       2023-10-30     8   $8.50     Auction  2021 Pokemon Celebrations   
 2       2023-10-27     9   $7.00     Auction  2021 Pokemon Celebrations   
 3       2023-10-18     9   $9.70     Auction  2021 Pokemon Celebrations   
 4       2023-10-06    10  $29.13     Auction  2021 Pokemon Celebrations   
 ...            ...   ...     ...         ...                        ...   
 543178  2018-05-02    10   $3.13     Auction         2001 Neo Discovery   
 543179  2018-02-26    10  $14.99     Auction         2001 Neo Discovery   
 543180  2018-02-03    10  $18.50     Auction         2001 Neo Discovery   
 543181  2017-06-23    10  $19.99  Buy It Now         2001 Neo Discovery   
 543182  2017-06-19    10  $24.99     Auction         2001 Neo Discovery   
 
                   card_name  
 0                     HO-Oh  
 1                     H

In [9]:
#returns and saves to xlsx the prices dataset in the long format:

import pandas as pd
import json
import os

def load_and_save_prices_data(sets_dir, output_file):
    """
    Loads price data from JSON files in the specified directory, 
    flattens it into a DataFrame, and saves it to an Excel file.

    Args:
    sets_dir (str): The directory containing set folders with JSON files.
    output_file (str): The path of the output Excel file to save the data.

    The function iterates through each set folder, reads each JSON file, 
    and compiles the price data into a single DataFrame. Each row in the 
    DataFrame represents a price record, with columns for set name, card name, 
    date, grade, price, and sale type. The DataFrame is then saved to the 
    specified Excel file.
    """
    all_prices = []

    # Iterate through each set and load price data
    all_sets = [d for d in os.listdir(sets_dir) if os.path.isdir(os.path.join(sets_dir, d))]
    for set_folder in all_sets:
        set_path = os.path.join(sets_dir, set_folder)
        json_files = [f for f in os.listdir(set_path) if f.endswith('.json')]

        for json_file in json_files:
            with open(os.path.join(set_path, json_file), 'r', encoding='utf-8') as file:
                data = json.load(file)
                set_name = data['set_name'].strip()
                card_name = data['card_name'].strip()

                # Check and Flatten prices_data
                if data['prices_data'] and isinstance(data['prices_data'], list):
                    for price_data in data['prices_data']:
                        if price_data is not None:
                            price_data['set_name'] = set_name
                            price_data['card_name'] = card_name
                            all_prices.append(price_data)

    # Create DataFrame from prices data
    prices_df = pd.DataFrame(all_prices)

    # Save to Excel
    prices_df.to_excel(output_file, index=False)
    print(f"Price data saved to {output_file}")

    return prices_df

# Example usage
sets_dir = 'sets'  # Replace with your actual directory
output_file = 'card_prices_data.xlsx'  # Replace with your desired output file path
prices_df = load_and_save_prices_data(sets_dir, output_file)


Price data saved to card_prices_data.xlsx


In [7]:
#returns and saves to xlsx the grades data in a wide format:

import pandas as pd
import json
import os

def load_data_in_wide_format_and_save_to_excel(sets_dir, output_file):
    """
    Loads card data from JSON files in the specified directory, 
    transforms it into a wide-format DataFrame, and saves it to an Excel file.

    Args:
    sets_dir (str): The directory containing set folders with JSON files.
    output_file (str): The path of the output Excel file to save the data.

    The function iterates through each set folder, reads each JSON file, 
    and compiles the data into a single DataFrame. Each row in the DataFrame 
    represents a card, with columns for set name, card name, and counts for 
    each grade. The DataFrame is then saved to the specified Excel file.
    """
    all_cards_data = []

    # Iterate through each set and load data
    all_sets = [d for d in os.listdir(sets_dir) if os.path.isdir(os.path.join(sets_dir, d))]
    for set_folder in all_sets:
        set_path = os.path.join(sets_dir, set_folder)
        json_files = [f for f in os.listdir(set_path) if f.endswith('.json')]

        for json_file in json_files:
            with open(os.path.join(set_path, json_file), 'r', encoding='utf-8') as file:
                data = json.load(file)

                # Construct card data for each JSON file
                card_data = {
                    'set_name': data['set_name'].strip(),
                    'card_name': data['card_name'].strip()
                }

                # Add grade data
                for grade_data in data['population_report']:
                    grade, count = grade_data
                    card_data[grade] = count

                all_cards_data.append(card_data)

    # Create DataFrame and save to Excel
    df = pd.DataFrame(all_cards_data)
    df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")

    return df

# Example usage
sets_dir = 'sets'  # Replace with your actual directory
output_file = 'card_data_grades.xlsx'  # Replace with your desired output file path
load_data_in_wide_format_and_save_to_excel(sets_dir, output_file)


Data saved to card_data_grades.xlsx


,set_name,card_name,PSA10,PSA9,PSA8,PSA7,PSA6,PSA5,PSA4,PSA3,PSA2,PSA1
0,2021 Pokemon Celebrations,HO-Oh,65,255,92,19,3,0,2,0,0,1
1,2021 Pokemon Celebrations,Zekrom,55,174,89,7,2,0,0,0,0,1
2,2021 Pokemon Celebrations,Mew,83,485,261,29,7,2,1,0,0,2
3,2021 Pokemon Celebrations,Xerneas,28,171,83,12,3,0,0,0,0,1
4,2021 Pokemon Celebrations,Cosmog,53,149,72,9,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22568,2001 Neo Discovery,Unown F,7,13,2,1,1,0,1,0,1,0
22569,2001 Neo Discovery,Unown F 1st Edition,35,55,9,2,4,0,1,0,0,0
22570,2001 Neo Discovery,Unown M,7,11,5,1,0,0,0,0,0,0
22571,2001 Neo Discovery,Unown M 1st Edition,38,49,10,1,1,3,0,0,0,0
